# Inputs
- Name
- Description
- Trading Strategy
- Data Sources
- Chain 2, Mainnet01
- Prompt
- Improved Prompt
- Tool Calling
- Transaction Execution

In [1]:
import os
import json
import requests
from typing import Dict, List, Any, Optional, Union, Tuple

# LangChain imports
from langchain.agents import Tool, AgentExecutor, create_openai_functions_agent
from langchain.memory import ConversationBufferMemory
from langchain.schema import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
# Set your OpenAI API key
from langchain_core.agents import AgentFinish, AgentActionMessageLog
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get OpenAI API key from environment variables
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [2]:
API_DOCS = {

    # Token transfer
    "transfer": {
        "description": "Transfer tokens from one account to another",
        "required_params": [
            "tokenAddress",  # Token contract address
            "sender",        # Sender account
            "receiver",      # Receiver account
            "amount",        # Amount to transfer
            "chainId"        # Chain ID (0-19)
        ],
        "optional_params": [
            {"name": "meta", "description": "Additional metadata"},
            {"name": "gasLimit", "description": "Gas limit for transaction"},
            {"name": "gasPrice", "description": "Gas price for transaction"},
            {"name": "ttl", "description": "Transaction time-to-live"}
        ],
        "endpoint": "/transfer"
    },
    
    # Token swapping
    "swap": {
        "description": "Swap one token for another using Kaddex/EchoDEX",
        "required_params": [
            "tokenInAddress",  # Address of input token
            "tokenOutAddress", # Address of output token
            "account",         # Sender account
            "chainId"          # Chain ID (0-19)
        ],
        "conditional_params": [
            {"name": "amountIn", "description": "Amount to swap", "condition": "Either amountIn or amountOut must be provided"},
            {"name": "amountOut", "description": "Desired output amount", "condition": "Either amountIn or amountOut must be provided"}
        ],
        "optional_params": [
            {"name": "slippage", "description": "Maximum acceptable slippage"}
        ],
        "endpoint": "/swap"
    },
    
    # Token quote
    "quote": {
        "description": "Get price quotes for swapping tokens",
        "required_params": [
            "tokenInAddress",  # Address of input token
            "tokenOutAddress", # Address of output token
            "chainId"          # Chain ID (0-19)
        ],
        "conditional_params": [
            {"name": "amountIn", "description": "Input amount to get output quote", "condition": "Either amountIn or amountOut must be provided"},
            {"name": "amountOut", "description": "Desired output amount to get input quote", "condition": "Either amountIn or amountOut must be provided"}
        ],
        "response": {
            "amountIn": "Required input amount (when amountOut is provided)",
            "amountOut": "Expected output amount (when amountIn is provided)",
            "priceImpact": "Price impact percentage as a string"
        },
        "endpoint": "/quote"
    },
}

TOKENS = """
mainnet:
  coin:
    symbol: KDA
    name: KDA
    description: Native token of Kadena
    img: img/kda.svg
    color: "#4a9079"
    totalSupply: 1000000000
    precision: 12
    socials:
      - type: website
        url: https://www.kadena.io/
      - type: twitter
        url: https://twitter.com/kadena_io
      - type: discord
        url: https://discord.com/invite/kadena
      - type: github
        url: https://github.com/kadena-io

  arkade.token:
    symbol: ARKD
    name: Arkade
    description:
    img: img/ark.png
    color: "#cc66ff"
    precision: 12
    socials:
      - type: website
        url: https://www.arkade.fun/
      - type: twitter
        url: https://twitter.com/ArkadeFun

  free.maga:
    symbol: MAGA
    name: MAGA
    description:
    img: img/maga.png
    color: "#9d0b32"
    precision: 12
    socials:
      - type: twitter
        url: https://x.com/MAGA_KDA

  free.crankk01:
    symbol: CRKK
    name: CRKK
    description:
    img: img/crankk.png
    color: "#7f6afc"
    precision: 12
    socials:
      - type: website
        url: https://crankk.io/


  free.cyberfly_token:
    symbol: CFLY
    name: CFLY
    description:
    img: img/cfly.svg
    color: "#1f1fc2"
    precision: 8
    socials: []

  free.finux:
    symbol: FINX
    name: FINUX
    description:
    img: img/finux.png
    color: "#23a45c"
    precision: 12
    socials: []

  free.kishu-ken:
    symbol: KISHK
    name: KISHK
    description: First Kadena memecoin 
    img: img/kishk.png
    color: "#cbcbcc"
    totalSupply: 1000000000000000.00
    circulatingSupply: 689488206446005.00
    precision: 12
    socials:
      - type: website
        url: https://kishuken.me/
      - type: twitter
        url: https://x.com/kishu_ken_kda
      - type: telegram
        url: https://t.me/kishukens
      
  kaddex.kdx:
    symbol: KDX
    name: KDX
    description: Kaddex / Ecko Token
    img: img/kdx.svg
    color: "#ff5271"
    totalSupply: 900699352.80
    circulatingSupply: 244,760,172.96
    precision: 12
    socials:
      - type: website
        url: https://ecko.finance/
      - type: github
        url: https://github.com/eckoDAO-org
      - type: twitter
        url: https://x.com/eckoDAO
      - type: discord
        url: https://discord.gg/eckodao

  n_625e9938ae84bdb7d190f14fc283c7a6dfc15d58.ktoshi:
    symbol: KTO
    name: KTO
    description: Katoshi
    img: img/ktoshi.png
    color: "#34daa8"
    precision: 15
    socials:
      - type: website
        url: https://ktoshi.com/
      - type: twitter
        url: https://x.com/ktoshis

  n_b742b4e9c600892af545afb408326e82a6c0c6ed.zUSD:
    symbol: zUSD
    name: zUSD
    description: Stable coin issued by Zelcore
    img: img/zUSD.svg
    color: "#8a62eb"
    precision: 18
    socials:
      - type: website
        url: https://zelcore.io/

  n_e309f0fa7cf3a13f93a8da5325cdad32790d2070.heron:
    symbol: HERON
    name: HERON
    description:
    img: img/heron.png
    totalSupply: 963142522
    circulatingSupply: 693142522
    color: "#a22726"
    precision: 12
    socials:
      - type: website
        url: https://www.heronheroes.com
      - type: twitter
        url: https://x.com/HeronHeroesKDA

  n_582fed11af00dc626812cd7890bb88e72067f28c.bro:
    symbol: BRO
    name: BRO
    description: Token of the Brother's Telegram group
    img: img/bro.png
    color: "#af826a"
    totalSupply: 100
    circulatingSupply: 80
    precision: 12
    socials:
        - type: website
          url: https://bro.pink/
        - type: twitter
          url: https://x.com/thebrothersdao

  runonflux.flux:
    symbol: FLUX
    name: FLUX
    description: Native token of the Flux blockchain
    img: img/flux-crypto.svg
    color: "#2b61d1"
    totalSupply: 440000000
    precision: 8
    socials:
      - type: website
        url: https://runonflux.io/
      - type: twitter
        url: https://t.me/zelhub
      - type: discord
        url: https://discord.gg/keVn3HDKZw

  free.wiza:
      symbol: WIZA
      name: WIZA
      description: Wizards Arena
      img: img/wizards.png
      color: "#ed0404"
      precision: 12
      socials:
        - type: website
          url: https://www.wizardsarena.net

  hypercent.prod-hype-coin:
    symbol: HYPE
    name: HYPE
    description: Hypercent token
    img: img/hypercent-crypto.svg
    color: "#c40a8d"
    totalSupply: 10000000
    precision: 12
    socials:
      - type: website
        url: https://hypercent.io/
      - type: twitter
        url: https://twitter.com/hypercentpad
      - type: discord
        url: https://discord.gg/dxVvdNhqaE
      - type: telegram
        url: http://t.me/HyperCent

  free.babena:
    symbol: BABE
    name: BABE
    description: Babena - First DEFI project on Kadena
    img: img/babena-logo.svg
    color: "#ffcc4d"
    totalSupply: 12967695
    precision: 12
    socials:
      - type: website
        url: https://babena.finance

  kdlaunch.token:
    symbol: KDL
    name: KDL
    description: KDLaunch
    img: img/kdl.svg
    color: "#4aa5b1"
    totalSupply: 100000000
    precision: 12
    socials:
      - type: website
        url: https://www.kdlaunch.com/
      - type: twitter
        url: https://twitter.com/KdLaunch
      - type: telegram
        url: https://t.me/KDLaunchOfficial
      - type: discord
        url: https://discord.com/invite/GghUdhmk6z

  kdlaunch.kdswap-token:
    symbol: KDS
    name: KDS
    description: KDSwap
    img: img/kds.svg
    color: "#6ebbf2"
    totalSupply: 100000000
    precision: 12
    socials:
      - type: website
        url: https://www.kdswap.exchange/
      - type: twitter
        url: https://twitter.com/KDSwap
      - type: telegram
        url: https://t.me/KDSwapOfficial
      - type: discord
        url: https://discord.com/invite/GghUdhmk6z

  n_2669414de420c0d40bbc3caa615e989eaba83d6f.highlander:
    symbol: HLR
    name: HLR
    description:
    img: img/uno.webp
    totalSupply: 1
    circulatingSupply: 1
    color: "#3d3939"
    precision: 12
    socials:
      - type: website
        url: https://youtu.be/dQw4w9WgXcQ?si=h0SS4HbaWxLgw2IA
  
  n_c89f6bb915bf2eddf7683fdea9e40691c840f2b6.cwc:
    symbol: CWC
    name: CWC
    description:
    img: img/cwc.webp
    totalSupply: 4000000
    circulatingSupply: 520
    color: "#a22726"
    precision: 12
    socials:
      - type: website
        url: guardiansofkadena.com
      - type: twitter
        url: https://x.com/GuardiansofKDA

  n_95d7fe012aa7e05c187b3fc8c605ff3b1a2c521d.MesutÖzilDönerKebabMerkel42Inu:
    symbol: KEBAB
    name: KEBAB
    description: This Token is a symbol of love to Döner Kebab and to the friendship between Germany and Turkey
    img: img/kebab.webp
    totalSupply: 100000000
    circulatingSupply: 100000000
    color: "#a22726"
    precision: 12
    socials: []
             
  n_95d7fe012aa7e05c187b3fc8c605ff3b1a2c521d.ShrekYodaTrumpMarsX12Inu:
    symbol: GREENCOIN
    name: GREENCOIN
    description: Cult for green coin, Trump and mars lovers.
    img: img/greencoin.webp
    totalSupply: 100000000
    circulatingSupply: 100000000
    color: "#a22726"
    precision: 12
    socials: []

  n_95d7fe012aa7e05c187b3fc8c605ff3b1a2c521d.SonGokuBezosPikachu12Inu:
    symbol: WLONG
    name: WLONG
    description: May the power of Wenlong be with us.
    img: img/wlong.webp
    totalSupply: 100000000
    circulatingSupply: 100000000
    color: "#a22726"
    precision: 12
    socials: []

  n_d8d407d0445ed92ba102c2ce678591d69e464006.TRILLIONCARBON:
    symbol: TCTC
    name: TCTC
    description: the official corporate token and ledger of Trillion Capital Toronto Corporation used for internal purposes
    img: img/tril.png
    totalSupply: 1000001
    circulatingSupply: 1000001
    color: "#a22726"
    precision: 12
    socials: 
      - type: website
        url: https://trillioncapital.ca
      - type: twitter
        url: https://twitter.com/TRILLIONCAP

  n_518dfea5f0d2abe95cbcd8956eb97f3238e274a9.AZUKI:
    symbol: AZUKI
    name: AZUKI
    description: Will Martino's beloved companion, AZUKI is a community managed token. Woof!.
    img: img/azuki.png
    totalSupply: 100000000
    circulatingSupply: 100000000
    color: "#218dc5"
    precision: 12
    socials:
      - type: website
        url: https://www.azukionkadena.fun
      - type: twitter
        url: https://x.com/AzukiKDA
      - type: telegram
        url: https://t.me/AzukiKDA

  n_71c27e6720665fb572433c8e52eb89833b47b49b.Peppapig:
    symbol: PP
    name: PP
    description:
    img: img/peppa.png
    totalSupply: 1000000000
    circulatingSupply: 1000000000
    color: "#a22726"
    precision: 12
    socials: 
      - type: telegram
        url: https://t.me/peppapigmemetokenkda

testnet:
  coin:
    symbol: KDA
    name: KDA
    description: Native token of Kadena
    img: img/kda.svg
    totalSupply: 1000000000
    socials:
      - type: website
        url: https://www.kadena.io/
      - type: twitter
        url: https://twitter.com/kadena_io
      - type: discord
        url: https://discord.com/invite/kadena
      - type: github
        url: https://github.com/kadena-io

blacklist:
  - lago.USD2
  - lago.kwBTC
  - lago.kwUSDC
  - free.elon
  - mok.token
  - free.docu
  - free.kpepe
  - free.backalley
  - free.kapybara-token
  - free.jodie-token
  - free.corona-token
  - free.KAYC
  - free.anedak
  - n_95d7fe012aa7e05c187b3fc8c605ff3b1a2c521d.MesutÖzilDönerKebabMerkel42Inu



"""

OUTPUT_FORMAT = {
  "rating": "<1–10>",
  "justification": "<one-sentence explanation of your score>",
  "questions": [
    "Question 1…",
    "Question 2…",
    "..."
  ]
}

In [3]:
model = ChatOpenAI(model="o4-mini")

In [4]:
history = []

In [5]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """
You are <Agent K0>, a trading agent launcher created by Xade.

You are tasked with helping users create prompts to launch trading agents on the Kadena blockchain.

You will be called repeatedly until the prompt is acceptable. Each time you receive:
  - A full draft of the user's system prompt (their new version or their previous one along with answers to your questions)
  - Any previous dialogue about the prompt
  - The same context inputs:
    - Agent Name
    - Agent Description
    - Trading Strategy

Based on the same, you will assing a rating to the prompt on a scale of 1-10.
Additionally, you will provide a list of questions that the user should address with the prompt.
     
Previous Dialogue:
{HISTORY}

Here are some resources to help you in your task:
  1. Documentation for Tokens:
    {TOKENS}
  This documentation contains information about all the tokens on the Kadena blockchain, so you can validate any on-chain addresses or symbols the user provides.
  2. Onchain Information:
  The Kadena blockchain (mainnet01) will be used on Chain ID 2. The DEX used will be Agent K, a custom DEX built by Xade. Do not ask questions about this.

When you are provided the prompt:
1. Evaluate the draft prompt for clarity and fitness to its specific strategy (from simple DCA to complex trading strategies).  
2. Assign a score (1–10) based only on clarity of intent and requirements.  
3. Justify your score in one concise sentence.  
4. Ask only the follow-up questions necessary to fill real gaps, including but not limited to:
   - Capital allocation (per-trade size, total risk budget)  
   - Entry/exit rules (indicators, thresholds, timeframes)  
   - Risk management (stop-loss, take-profit, max drawdown)  
   - Order types (market/limit/conditional), slippage, fee assumptions  

Output Format: 
> - Output Structured JSON with only the following keys:
> - rating (number between 1 and 10)
> - justification (one sentence explanation of your score)
> - questions (list of questions)

> **Notes:**
> - Authentication and transaction signing is handled later; omit related questions.  
> - All the handling of execution-failure and other such issues is handled directly by Xade. Do not bother the user with such issues.
> - Avoid over-engineering: for simple strategies, skip irrelevant details.  
> - Be consistent with your ratings.
"""),
    ("human", "{input}")
])

In [6]:
query = "A DCA trading agent that buys 100 zUSD using KDA every 30 minutes at market price."
response = model.invoke(prompt.format(input=query, TOKENS=TOKENS, OUTPUT_FORMAT=OUTPUT_FORMAT, HISTORY=history))
history.extend([
        "Human: "+query,
        "AI: "+str(response.content)
    ])

In [7]:
response.content

'{"rating":7,"justification":"The DCA plan is clear on amount and frequency but omits total budget, runtime limits, and slippage/risk guardrails.","questions":["What is your total KDA budget or maximum number of 100 zUSD purchases the agent may execute?","Should the agent run indefinitely or stop after a specific time period or number of trades?","What slippage tolerance (e.g. % deviation) should be applied to each market order?","Do you want any overall exposure limit or stop conditions (e.g. max zUSD balance or stop if price moves >X%)?"]}'

In [8]:
query = "Budget is until the amount of KDA in my wallet runs out. Slippage Tolerance is 0.005. No max drawdown."
response = model.invoke(prompt.format(input=query, TOKENS=TOKENS, OUTPUT_FORMAT=OUTPUT_FORMAT, HISTORY=history))
history.extend([
        "Human: "+query,
        "AI: "+str(response.content)
    ])

In [9]:
response.content

'{"rating":8,"justification":"The prompt clearly specifies DCA size, frequency, total budget mechanism, and slippage tolerance, but omits handling of transaction fees and partial‐fund scenarios.","questions":["Should the agent maintain a minimum KDA balance to cover network fees before submitting each order?","If your remaining KDA cannot fund a full 100 zUSD purchase, should the agent place a partial order or stop trading?","Do you want the agent to operate continuously 24/7 or restrict trading to certain hours?"]}'

In [10]:
query = "Fee treatment is separate from the price of 100 zUSD."
response = model.invoke(prompt.format(input=query, TOKENS=TOKENS, OUTPUT_FORMAT=OUTPUT_FORMAT, HISTORY=history))
history.extend([
        "Human: "+query,
        "AI: "+str(response.content)
    ])

In [11]:
response.content

'{"rating":8,"justification":"The strategy now clearly defines trade size, frequency, total budget behavior, slippage tolerance, and fee treatment, but still lacks handling for partial‐fill scenarios and operating schedule.","questions":["Should the agent reserve KDA separately to cover network fees so that each order uses exactly 100 zUSD worth of KDA?","If your remaining KDA cannot fund a full 100 zUSD purchase, should the agent place a smaller partial order or stop trading entirely?","Do you want the agent to trade continuously 24/7 or restrict it to specific hours or days?"]}'

In [12]:
TRANSACTIONS_CODE = """
/**
 * @description JavaScript client for Kadena blockchain API operations
 * 
 * This module provides functions to interact with Kadena blockchain through a REST API,
 * allowing token transfers, swaps, and price quotes.
 */

// Base URL for the API (should be configured based on environment)
const API_BASE_URL = "https://kadena-agents.onrender.com";
// API key for authentication (should be provided by the user)
const API_KEY = "YOUR_API_KEY"; 

/**
 * Sets the API key for authentication
 * @param {string} apiKey - The API key to use for authentication
 */
function setApiKey(apiKey) {
  if (!apiKey || typeof apiKey !== "string") {
    throw new Error("Invalid API key provided");
  }
  this.API_KEY = apiKey;
}

/**
 * Makes a request to the Kadena API
 * @param {string} endpoint - The API endpoint to call
 * @param {Object} body - The request body containing parameters
 * @returns {Promise<Object>} The API response
 * @private
 */
async function makeRequest(endpoint, body) {
  try {
    const response = await fetch(`${API_BASE_URL}${endpoint}`, {
      method: "POST",
      headers: {
        "Content-Type": "application/json",
        "x-api-key": API_KEY
      },
      body: JSON.stringify(body)
    });

    if (!response.ok) {
      const errorData = await response.json().catch(() => ({ error: "Unknown error" }));
      throw new Error(`API Error (${response.status}): ${errorData.error || response.statusText}`);
    }

    return await response.json();
  } catch (error) {
    throw new Error(`Request failed: ${error.message}`);
  }
}

/**
 * Validates a chain ID parameter
 * @param {string|number} chainId - The chain ID to validate
 * @returns {string} The validated chain ID as a string
 * @private
 */
function validateChainId(chainId) {
  const chainIdStr = String(chainId);
  const chainIdNum = parseInt(chainIdStr, 10);
  
  if (isNaN(chainIdNum) || chainIdNum < 0 || chainIdNum > 19) {
    throw new Error("Chain ID must be between 0 and 19");
  }
  
  return chainIdStr;
}

/**
 * Transfer tokens from one account to another
 * 
 * @param {Object} params - Transfer parameters
 * @param {string} params.tokenAddress - Token contract address
 * @param {string} params.sender - Sender account
 * @param {string} params.receiver - Receiver account
 * @param {string|number} params.amount - Amount to transfer
 * @param {string|number} params.chainId - Chain ID (0-19)
 * @param {Object} [params.meta] - Additional metadata
 * @param {number} [params.gasLimit] - Gas limit for transaction
 * @param {number} [params.gasPrice] - Gas price for transaction
 * @param {number} [params.ttl] - Transaction time-to-live
 * @returns {Promise<Object>} Transfer transaction data containing:
 *   - transaction: Object containing:
 *      - cmd: The Pact command JSON string
 *      - hash: Transaction hash
 *      - sigs: Array of signatures (null if not signed)
 *   - metadata: Object containing:
 *      - sender: Sender account
 *      - receiver: Receiver account
 *      - amount: Transfer amount
 *      - tokenAddress: Token contract address
 *      - chainId: Chain ID
 *      - networkId: Network ID (e.g., "mainnet01")
 *      - estimatedGas: Estimated gas cost
 *      - formattedAmount: Formatted transfer amount
 */
async function transfer({
  tokenAddress,
  sender,
  receiver,
  amount,
  chainId,
  meta,
  gasLimit,
  gasPrice,
  ttl
}) {
  // Validate required parameters
  if (!tokenAddress) throw new Error("tokenAddress is required");
  if (!sender) throw new Error("sender is required");
  if (!receiver) throw new Error("receiver is required");
  if (amount === undefined || amount === null) throw new Error("amount is required");
  
  // Validate and format chainId
  const validatedChainId = validateChainId(chainId);
  
  // Prepare request body
  const requestBody = {
    tokenAddress,
    sender,
    receiver,
    amount: String(amount),
    chainId: validatedChainId
  };
  
  // Add optional parameters if provided
  if (meta !== undefined) requestBody.meta = meta;
  if (gasLimit !== undefined) requestBody.gasLimit = gasLimit;
  if (gasPrice !== undefined) requestBody.gasPrice = gasPrice;
  if (ttl !== undefined) requestBody.ttl = ttl;
  
  // Make API request
  return await makeRequest("/transfer", requestBody);
}

/**
 * Swap one token for another using Kaddex/EchoDEX
 * 
 * @param {Object} params - Swap parameters
 * @param {string} params.tokenInAddress - Address of input token
 * @param {string} params.tokenOutAddress - Address of output token
 * @param {string} params.account - Sender account
 * @param {string|number} params.chainId - Chain ID (0-19)
 * @param {string|number} [params.amountIn] - Amount to swap (either amountIn or amountOut must be provided)
 * @param {string|number} [params.amountOut] - Desired output amount (either amountIn or amountOut must be provided)
 * @param {number} [params.slippage] - Maximum acceptable slippage
 * @returns {Promise<Object>} Swap transaction data containing:
 *   - transaction: Object containing:
 *      - cmd: The Pact command JSON string with swap details
 *      - hash: Transaction hash
 *      - sigs: Array of signatures (null if not signed)
 *   - quote: Object containing:
 *      - expectedIn: The exact input amount
 *      - expectedOut: The expected output amount
 *      - slippage: Applied slippage tolerance
 *      - priceImpact: Price impact percentage
 */
async function swap({
  tokenInAddress,
  tokenOutAddress,
  account,
  chainId,
  amountIn,
  amountOut,
  slippage
}) {
  // Validate required parameters
  if (!tokenInAddress) throw new Error("tokenInAddress is required");
  if (!tokenOutAddress) throw new Error("tokenOutAddress is required");
  if (!account) throw new Error("account is required");
  
  // Validate conditional parameters
  if (amountIn === undefined && amountOut === undefined) {
    throw new Error("Either amountIn or amountOut must be provided");
  }
  if (amountIn !== undefined && amountOut !== undefined) {
    throw new Error("Cannot specify both amountIn and amountOut");
  }
  
  // Validate and format chainId
  const validatedChainId = validateChainId(chainId);
  
  // Prepare request body
  const requestBody = {
    tokenInAddress,
    tokenOutAddress,
    account,
    chainId: validatedChainId
  };
  
  // Add conditional parameters
  if (amountIn !== undefined) requestBody.amountIn = String(amountIn);
  if (amountOut !== undefined) requestBody.amountOut = String(amountOut);
  
  // Add optional parameters if provided
  if (slippage !== undefined) requestBody.slippage = slippage;
  
  // Make API request
  return await makeRequest("/swap", requestBody);
}

/**
 * Get price quotes for swapping tokens
 * 
 * @param {Object} params - Quote parameters
 * @param {string} params.tokenInAddress - Address of input token
 * @param {string} params.tokenOutAddress - Address of output token
 * @param {string|number} params.chainId - Chain ID (0-19)
 * @param {string|number} [params.amountIn] - Input amount to get output quote (either amountIn or amountOut must be provided)
 * @param {string|number} [params.amountOut] - Desired output amount to get input quote (either amountIn or amountOut must be provided)
 * @returns {Promise<Object>} Quote response containing:
 *   - amountIn: Required input amount (when amountOut is provided)
 *   - amountOut: Expected output amount (when amountIn is provided)
 *   - priceImpact: Price impact percentage as a string
 */
async function quote({
  tokenInAddress,
  tokenOutAddress,
  chainId,
  amountIn,
  amountOut
}) {
  // Validate required parameters
  if (!tokenInAddress) throw new Error("tokenInAddress is required");
  if (!tokenOutAddress) throw new Error("tokenOutAddress is required");
  
  // Validate conditional parameters
  if (amountIn === undefined && amountOut === undefined) {
    throw new Error("Either amountIn or amountOut must be provided");
  }
  if (amountIn !== undefined && amountOut !== undefined) {
    throw new Error("Cannot specify both amountIn and amountOut");
  }
  
  // Validate and format chainId
  const validatedChainId = validateChainId(chainId);
  
  // Prepare request body
  const requestBody = {
    tokenInAddress,
    tokenOutAddress,
    chainId: validatedChainId
  };
  
  // Add conditional parameters
  if (amountIn !== undefined) requestBody.amountIn = String(amountIn);
  if (amountOut !== undefined) requestBody.amountOut = String(amountOut);
  
  // Make API request
  return await makeRequest("/quote", requestBody);
}
"""

TRANSACTIONS_USAGE = """
// Example 1: Transfer KDA tokens
  try {
    const result = await kadenaApi.transfer({
      tokenAddress: 'coin', // 'coin' is the KDA token
      sender: 'k:sender_account_key',
      receiver: 'k:receiver_account_key',
      amount: '10.0',
      chainId: '2'
    });
    console.log('Transfer transaction:', result);
  } catch (error) {
    console.error('Transfer failed:', error.message);
  }


// Example 2: Get a quote for swapping tokens
  try {
    const result = await kadenaApi.quote({
      tokenInAddress: 'coin', // KDA
      tokenOutAddress: 'kaddex.kdx', // KDX token
      amountIn: '100.0',
      chainId: '2'
    });
    console.log('Swap quote:', result);
    console.log(`Expected output: ${result.amountOut}`);
    console.log(`Price impact: ${result.priceImpact}`);
  } catch (error) {
    console.error('Quote failed:', error.message);
  }


// Example 3: Execute a token swap
  try {
    const result = await kadenaApi.swap({
      tokenInAddress: 'coin', // KDA
      tokenOutAddress: 'kaddex.kdx', // KDX token
      account: 'k:account_key',
      amountIn: '50.0',
      slippage: 0.01, // 1% slippage tolerance
      chainId: '2'
    });
    console.log('Swap transaction:', result);
  } catch (error) {
    console.error('Swap failed:', error.message);
  }


// Example 4: Sign and Submit a transaction
  try {
    const result = await kadenaApi.swap({
      tokenInAddress: 'coin', // KDA
      tokenOutAddress: 'kaddex.kdx', // KDX token
      account: 'k:account_key',
      amountIn: '50.0',
      slippage: 0.01, // 1% slippage tolerance
      chainId: '2'
    });
    console.log('Swap transaction:', result);
  } catch (error) {
    console.error('Swap failed:', error.message);
  }  

*/
"""

In [13]:
BASELINE_JS = """
[CODE]
// Baseline function for Kadena blockchain transactions
// This code provides the infrastructure for:
// 1. Retrieving keys from AWS KMS
// 2. Transaction signing
// 3. Transaction submission
// The AI model should focus on implementing the transaction creation logic

const { KMSClient, DecryptCommand } = require("@aws-sdk/client-kms");
const { createHash } = require("crypto");
const { sign } = require("@kadena/cryptography-utils");
const { Pact } = require("@kadena/client");
require("dotenv").config();

// Initialize AWS KMS client
const kmsClient = new KMSClient({
  region: process.env.AWS_REGION,
  credentials: {
    accessKeyId: process.env.AWS_ACCESS_KEY_ID,
    secretAccessKey: process.env.AWS_SECRET_ACCESS_KEY,
  },
});

/**
 * Retrieves and decrypts the private key from AWS KMS
 */
async function getKeysFromKMS() {
  try {
    if (!process.env.ENCRYPTED_PRIVATE_KEY) {
      throw new Error("ENCRYPTED_PRIVATE_KEY environment variable is not set");
    }

    const command = new DecryptCommand({
      CiphertextBlob: Buffer.from(process.env.ENCRYPTED_PRIVATE_KEY, "base64"),
    });

    const response = await kmsClient.send(command);
    const privateKey = response.Plaintext.toString("utf-8");

    // Validate the private key format
    if (!/^[0-9a-f]{64}$/.test(privateKey)) {
      throw new Error(
        `Invalid private key format - should be 64 hex characters, got length ${privateKey.length}`
      );
    }

    // Return the key pair
    return {
      secretKey: privateKey,
      publicKey:
        "38c0944b62d06a1c16fde2556a5e2ee3872efe9095e0050c8d16819f7306d382", // This should be derived from the private key
    };
  } catch (error) {
    console.error("Error retrieving keys from KMS:", error);
    throw new Error(`Failed to retrieve keys from KMS: ${error.message}`);
  }
}

/**
 * Signs a transaction using the provided key pair
 */
async function signTransaction(transaction, keyPair) {
  try {
    // Convert transaction to string if it's not already
    const txString =
      typeof transaction === "string"
        ? transaction
        : JSON.stringify(transaction);

    // Create a hash of the transaction
    const hash = transaction.hash;

    // Sign the hash with the key pair
    const signature = sign(hash, keyPair);

    return signature;
  } catch (error) {
    console.error("Error signing transaction:", error);
    throw new Error(`Failed to sign transaction: ${error.message}`);
  }
}

/**
 * Submits a signed transaction to the Kadena blockchain
 */
async function submitTransaction(signedTransaction) {
  try {
    // Create a Pact transaction
    const pact = new Pact();

    // TODO: Configure the Pact transaction with the signed transaction data
    // This will be filled in by the AI model
    console.log(
      "Submitting transaction to Kadena blockchain:",
      signedTransaction
    );

    // Mock response for now
    return {
      success: true,
      transactionId: "mock-transaction-id",
    };
  } catch (error) {
    console.error("Error submitting transaction:", error);
    throw new Error(`Failed to submit transaction: ${error.message}`);
  }
}

/**
 * Main baseline function that orchestrates the entire process
 */
async function baselineFunction() {
  try {
    // 1. Retrieve keys from KMS
    console.log("Retrieving keys from KMS...");
    const keyPair = await getKeysFromKMS();
    console.log("Keys retrieved successfully");

    // 2. Create transaction (placeholder)
    console.log("Creating transaction...");

    // ENTER CODE HERE

    console.log("Transaction created:", transaction);

    // 3. Sign the transaction
    console.log("Signing transaction...");
    const signature = await signTransaction(transaction, keyPair);
    console.log("Transaction signed successfully");

    // 4. Submit the transaction
    console.log("Submitting transaction...");
    const result = await submitTransaction({
      ...transaction,
      signature,
    });
    console.log("Transaction submitted successfully:", result);

    return result;
  } catch (error) {
    console.error("Error in baseline function:", error);
    throw error;
  }
}

// Example usage
if (require.main === module) {
  baselineFunction()
    .then((result) => console.log("Baseline function completed:", result))
    .catch((error) => console.error("Baseline function failed:", error));
}

module.exports = {
  baselineFunction,
  getKeysFromKMS,
  signTransaction,
  submitTransaction,
};
[/CODE]
"""

In [14]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """
    You are <Agent K1>, a trading agent launcher created by Xade.

    Your task is to generate code to run on a serverless function to execute a user's trading positions on the Kadena Blockchain.
    You will be wokring only on mainnet01 and chain ID 2.
    You will be writing code in JavaScript.
    
    You will be provided with a prompt containing all the information required to handle and execute the trading position.
    You will be provided with the user's account details. You will also be provided with the balances of all the user's tokens.
    You will have access to all the functions you may need to include to achieve this task as well.

    Here are some resources to help you in your task:
      1. Code for Transactions:
        {TRANSACTIONS_CODE}
        This code contains the code to call the Transactions API to generate unsigned transactions. THese functuons will be pre-defined. You need to use them to generate transactions.
        The above snippet also contains the documentation for the various functions.
      2. Transactions Usage:
        {TRANSACTIONS_USAGE}
        This contains examples to call/access the various endpoints of the Transactions API, code to sign and submit transactions and other functions for which the documentation is provided above.
      3. Documentation for Tokens:
        {TOKENS}
        This documentation contains information about all the tokens on the Kadena Blockchain.

    When a user prompt arrives:
    1. Analyze requirements:
      - Use the TOKENS documentation to validate any symbols or coins or addresses that the user provides.
      - Use the TRANSACTIONS CODE to understand the required functions and parameters.
      - Analyze the user's prompt to understand the steps required to execute the trading position.
      - Create a step-by-step plan to execute the trading position.
    2. Generate code:
      - Create a function for each step in the plan.
      - Hardcode all the parameters for each function based on the requirements, since all the parameters are known.
      - Use logic and knowledge of JavaScript syntax to write high-quality, efficient code for each function.
      - Use the TRANSACTIONS USAGE to understand how to call the various functions.
      - Input the code to create the transaction in the baseline.js file provided. Input it in the area de-marked for you to do so.
      - Do not change any other code in the baseline.js file. You can define variables wherever you want.

      - Special Case:
        a) If the user asks you for the value or price of a token, use the quotes transaction tool to get the price of the token.
        b) if the user asks for a value of any token, return it in terms of KDA and if they ask for vlaue of KDA, return in terms of zUSD.
    
    3. Always:
      - Think step-by-step before responding (internally).
      - Output the entire file onwards from the baselineFunction() definition.
     
     BASELINE.JS:
     {BASELINE_JS}
     
     > **Notes:**
          > - The user will not be involved in the execution. Thus, you must write impeccable code.
          > - Avoid over-engineering: keep the code simple yet effective.
          > - Whenever USD is mentioned, assume it is zUSD.
          > - Output only the code. No other text.
          > - Hardcode the time interval as a variable `timeInterval` in ms that is exported.
          > - Do not implement the continous execution logic. That will be handled by the AWS Lambda function.
     
    """),
    ("human", "{input}")
])

In [15]:
query = "A DCA trading agent that buys 1 USD using KDA every 30 minutes at market price if the value of 1KDA is greater than 0.6USD. Budget is until the amount of KDA in my wallet runs out. Slippage Tolerance is 0.005. No max drawdown. Fee treatment is separate from the budget."
response = model.invoke(prompt.format(input=query, TOKENS=TOKENS, TRANSACTIONS_CODE=TRANSACTIONS_CODE, TRANSACTIONS_USAGE=TRANSACTIONS_USAGE, BASELINE_JS=BASELINE_JS))

In [16]:
print(response.content)

```javascript
async function baselineFunction() {
  try {
    // 1. Retrieve keys from KMS
    console.log("Retrieving keys from KMS...");
    const keyPair = await getKeysFromKMS();
    console.log("Keys retrieved successfully");

    // 2. Create transaction
    console.log("Creating transaction...");

    // DCA parameters
    const chainId = "2";
    const tokenInAddress = "coin"; // KDA
    const tokenOutAddress = "n_b742b4e9c600892af545afb408326e82a6c0c6ed.zUSD"; // zUSD
    const amountIn = "1.0";    // 1 KDA for price check
    const amountOut = "1.0";   // target 1 zUSD per trade
    const slippage = 0.005;    // 0.5% slippage tolerance
    let transaction;

    // 2a. Get price quote for 1 KDA in zUSD
    console.log("Fetching price quote for 1 KDA in zUSD...");
    const quoteResult = await quote({
      tokenInAddress,
      tokenOutAddress,
      amountIn,
      chainId,
    });
    const priceZUSD = parseFloat(quoteResult.amountOut);
    console.log(`Price for 1 KDA is ${

In [17]:
import re

def replace_from_baseline_function(original_code: str, updated_tail: str) -> str:
    pattern = r"(async function baselineFunction\s*\(\)\s*{)"
    match = re.search(pattern, original_code)
    if not match:
        raise ValueError("Could not find baselineFunction() in the original code.")

    start_index = match.start()
    return original_code[:start_index] + updated_tail.strip()

In [18]:
print(replace_from_baseline_function(BASELINE_JS, response.content))


[CODE]
// Baseline function for Kadena blockchain transactions
// This code provides the infrastructure for:
// 1. Retrieving keys from AWS KMS
// 2. Transaction signing
// 3. Transaction submission
// The AI model should focus on implementing the transaction creation logic

const { KMSClient, DecryptCommand } = require("@aws-sdk/client-kms");
const { createHash } = require("crypto");
const { sign } = require("@kadena/cryptography-utils");
const { Pact } = require("@kadena/client");
require("dotenv").config();

// Initialize AWS KMS client
const kmsClient = new KMSClient({
  region: process.env.AWS_REGION,
  credentials: {
    accessKeyId: process.env.AWS_ACCESS_KEY_ID,
    secretAccessKey: process.env.AWS_SECRET_ACCESS_KEY,
  },
});

/**
 * Retrieves and decrypts the private key from AWS KMS
 */
async function getKeysFromKMS() {
  try {
    if (!process.env.ENCRYPTED_PRIVATE_KEY) {
      throw new Error("ENCRYPTED_PRIVATE_KEY environment variable is not set");
    }

    const comman